# Programming a P4 Switches
Programming a P4 switch, whether targeting hardware or software, requires a dedicated software development environment equipped with a compiler. 

**Figure 1: Generic workflow design**

![Workflow](./figs/p4-workflow.png)

The workflow consists of the following components:

1. **Vendor-Provided Components**:
   - **Compiler**: Responsible for translating the target-independent P4 source code into platform-specific implementations.
   - **Architecture Model**: Defines the capabilities and constraints of the target device.
   - **Target Switch**: The hardware or software environment where the data plane logic is deployed.

2. **User-Provided Component**:
   - **P4 Source Code**: Customized by the user to specify the desired forwarding logic and behavior.

### Compilation Process

The compiler translates the user-defined P4 source code into two essential artifacts:

1. **Data Plane Runtime Configuration**:  
   - Implements the forwarding logic defined in the P4 program.
   - Contains instructions and resource mappings tailored to the specific target switch.

2. **Runtime Application Programming Interfaces (APIs)**:  
   - Enable the control plane or user to interact with the data plane during runtime.
   - Support operations such as adding/removing entries in match-action tables and reading/writing the state of external objects (e.g., counters, meters, registers).
   - Provide the necessary information for the control plane to manipulate data plane components, including:
     - Table identifiers
     - Match fields
     - Keys
     - Action parameters

## Set Up a Network Slice with a P4 Tofino Switch and a Basic P4 Program

In this notebook, we will configure a network slice that utilizes a P4 Tofino switch to run a basic P4 program. The setup will establish connectivity between two nodes via the P4 switch, allowing them to communicate and test connectivity using `ping`.

This example is inspired by the **University of South Carolina CyberTraining program** for P4 programmable devices, which provides hands-on training on using P4 to build and manage network slices effectively.

## Prerequisites

1. **Access to a P4 Tofino switch**: A programmable hardware device equipped with an Intel Tofino ASIC.
2. **Two nodes for connectivity**: Ensure two nodes are physically or virtually connected to the P4 switch.
3. **Basic knowledge of P4 programming**: Familiarity with writing, compiling, and deploying P4 programs is necessary.
4. **Installed Tofino Software Development Environment (SDE)**: Required tools and libraries for P4 development and deployment.

## Example Workflow Overview

1. **Compile the P4 Program**:
   - Use the **bf-p4c compiler** with the **Tofino Native Architecture (TNA)** to compile the program into a binary file.
   - This binary will serve as the data plane runtime configuration for the P4 switch.

2. **Load the Data Plane Program**:
   - Load the compiled binary into the P4 switch daemon, ensuring the forwarding logic is active.

3. **Configure the Control Plane**:
   - Use the **bfrt-python API** to populate the match-action tables, specifying forwarding rules for the nodes.

4. **Test Connectivity**:
   - Use `ping` to test connectivity between the two nodes and validate that the network slice is functioning as intended.

**Figure 2: P4 example workflow**

![Lab Workflow](./figs/p4-lab-workflow.png)


**Figure 3: Slice Topology**

![Topology](./figs/p4-slice.png)

The topology depicts two nodes connected through the P4 switch. The P4 program running on the switch enables forwarding, allowing the nodes to communicate.


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following script sets up two nodes, each with Shared NICs connected to two ports on a P4 tofino switch.

NIC component model options include:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports)
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)

### Identify the Sites

List available sites with `P4 Switch`. Chooose one site at random from the available sites.

In [ ]:
p4_column_name = 'p4-switch_available'

'''
# Find a site which has a P4 Switch available
[site2] = fablib.get_random_sites(count=1, filter_function=lambda x: x[p4_column_name] > 0)

# Choose another random site other than P4 site to host the VMs
site1 = fablib.get_random_site(avoid=[site2])
'''
site2="STAR"
site1="SALT"

print(f"Sites chosen for hosting VMs: {site1} P4: {site2}")

### Define variables

In [ ]:
slice_name = 'P4-Lab-Slice2'
p4_column_name = "p4-switch_available"

node1_name = 'Node1'
node2_name = 'Node2'
p4_name = 'P4'
network1_name = 'net1'
network2_name = 'net2'

print(f"VM Site: {site1}")
print(f"P4 Site: {site2}")

#model='NIC_ConnectX_6'
model='NIC_Basic'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Create Network
net1 = slice.add_l2network(name=network1_name, subnet=IPv4Network("192.168.0.0/24"))
net2 = slice.add_l2network(name=network2_name, subnet=IPv4Network("192.168.0.0/24"))

# Create Node 1 and its links
node1 = slice.add_node(name=node1_name, site=site1)
iface1 = node1.add_component(model=model, name='nic1').get_interfaces()[0]
iface1.set_mode('config')
net1.add_interface(iface1)
iface1.set_ip_addr(IPv4Address("192.168.0.1"))

# Create P4 switch and its links 
p4 = slice.add_switch(name=p4_name, site=site2)
iface2 = p4.get_interfaces()[0]
iface3 = p4.get_interfaces()[1]

net1.add_interface(iface2)
net2.add_interface(iface3)

# Create Node 2 and its links
node2 = slice.add_node(name=node2_name, site=site1)
iface4 = node2.add_component(model=model, name='nic1').get_interfaces()[0]
iface4.set_mode('config')
net2.add_interface(iface4)
iface4.set_ip_addr(IPv4Address("192.168.0.2"))

# Submit Slice Request
slice.submit()

## Compilling the basic P4 program
P4 program file `basic.p4` is available inside a docker conatiner `p4_container` running on the switch and can be edited by the user via `vim`

**Use the SSH command provided above to connect to the P4 switch. Once connected, execute the commands below to access the SDE environment. The SDE is set up within the `p4_container` Docker container, so you will need to enter the container's bash shell.**

```
docker exec -ti p4_container /bin/bash
```

### Configure the environment variables

Go to Lab1 and source config environment
```
cd ~/P4_labs/lab1/
source config_env.sh
```


### Compile the code

In this example we will not modify the P4 code. Instead, we will just compile it and run it on the switch. 

The script `p4_build.sh` is provided by Intel’s support portal. It invokes the compiler to generate the output files. It automatically detects the P4 version (i.e., P416), and generates
the output files under `$SDE/build/p4-build/<program_name>`, where `<program_name>` is the file `basic.p4`. It also generates the log files under `$SDE/logs/p4-build/<program_name>` and other files (e.g., graphs).

Use the command below to compile the code:

```
~/tools/p4_build.sh --with-p4c=bf-p4c p4src/basic.p4
```

After executing the command, if there are no error messages displayed in the terminal, then the P4 program was compiled successfully.

### Verify the compilation output files
Verify the output files for the program basic.p4 are generated in the `$SDE/build/p4-build` directory by issuing the following command.

```
ls $SDE/build/p4-build/tofino/basic/
```

The binary file that corresponds to the compiled data plane is located under `ls $SDE/build/p4-build/tofino/<program_name>/<program_name>/tofino/pipe/`. Use the command below to display the contents of this directory.

```
ls $SDE/build/p4-build/tofino/basic/basic/tofino/pipe/
```

## Start the switch daemon
Now that we have compiled our P4 program and generated the output files, we can start the switch daemon with the compiled output using the command below.

```
cd ~/bf-sde-9.13.3
./run_switchd.sh -p basic
```

NOTE: If you see error regarding kernel module, please run the following command:
```
sudo $SDE_INSTALL/bin/./bf_kdrv_mod_load $SDE_INSTALL
```

## Configuring the switch ports

Once the daemon is running, you can issue commands in the bfshell.  

1. Issue the following command in the switch CLI to manage the ports of the switch.

```
ucli
```

NOTE: The ucli is the bfshell instance used to manage the switch ports. With the ucli, you can enable or disable ports, set the port speed (e.g., 100 Gbps, 40 Gbps, and 10 Gbps), and
select the FEC type. Additionally, the user can monitor the status of the ports, the number of sent and received frames, and other variables.

2. Now we will add the ports for the switch. Recall from the topology that Server1 is connected to port 1 on the switch, and Server2 is connected to port 2 on the switch. Issue the
commands below to add ports 1 and 2 in the Tofino switch.

```
pm port-add 1/- 100G NONE
pm port-add 2/- 100G NONE
```

3. Enable the ports by issuing the following commands.
```
pm port-enb 1/-
pm port-enb 2/-
```

4. Verify that the ports are up by issuing the following command.
```
pm show
```


**Expected output:**
```
bf-shell> ucli
bf-sde> pm port-add 1/- 100G NONE
bf-sde> pm port-add 2/- 100G NONE
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|132|3/ 4|100G   |NONE|Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
2/0  |22/0|140|3/12|100G   |NONE|Au|Au|YES|ENB|DWN|  NONE  |               0|               0|
bf-sde> pm port-enb 1/-
bf-sde> pm port-enb 2/-
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|132|3/ 4|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              29|              14|
2/0  |22/0|140|3/12|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              29|              14|
```

The ouput above shows that ports 1 and 2 are up. The speed for the ports is 100G. The columns `FRAMES TX` indicate how many frames are received and `FRAMES RX` transmitted in each port respectively.

## Populating the switch’s forwarding table

**Use the SSH command provided above to connect to the P4 switch. Once connected, execute the commands below to access the SDE environment. The SDE is set up within the `p4_container` Docker container, so you will need to enter the container's bash shell.**

```
docker exec -ti p4_container /bin/bash
```

**1. Update the port numbers highligted in red box in `~/P4_labs/lab1/bfrt_python/setup.py`. Port numbers should match the `D_P` column as observed in the output of `pm show` above.**

<img src="./figs/p4-brft-setup.png" alt="brft" style="width:40%;">





**2. Populate the table entries to the switch by typing the following command in the embedded terminal.**

```
cd ~/bf-sde-9.13.3; . ~/tools/set_sde.bash
$SDE/./run_bfshell.sh --no-status-srv -b ~/P4_labs/lab1/bfrt_python/setup.py
```

The forwarding table contains the information that the P4 program will use to forward packets to the right destination. 

## Verifying Reachability through Ping Tests

Inititate ping between the VMs.

In [ ]:
slice=fablib.get_slice(slice_name)
node1=slice.get_node(node1_name)
node2=slice.get_node(node2_name)

node1_addr = node1.get_interface(network_name=network1_name).get_ip_addr()
node2_addr = node2.get_interface(network_name=network2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')
stdout, stderr = node2.execute(f'ping -c 5 {node1_addr}')

### Verify frame counters on the switch

Now, verify the counters for the ports on the switch.
Both RX and TX frame counts should be incremented.

```
bf-sde> pm show
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
PORT |MAC |D_P|P/PT|SPEED  |FEC |AN|KR|RDY|ADM|OPR|LPBK    |FRAMES RX       |FRAMES TX       |E
-----+----+---+----+-------+----+--+--+---+---+---+--------+----------------+----------------+-
1/0  |23/0|128|2/ 0|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              61|              13|
2/0  |22/0|136|2/ 8|100G   |NONE|Au|Au|YES|ENB|UP |  NONE  |              29|              17|
```

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice=fablib.get_slice(slice_name)
slice.delete()